# Recraft V3 Image Generator

This notebook connects to Recraft V3 via Replicate API to generate new variations of your branding and icon designs.

## Setup Instructions

1. Create a `.env` file in the same directory as this notebook
2. Add your Replicate API token: `REPLICATE_API_TOKEN=your_token_here`
3. Get your token from: https://replicate.com/account/api-tokens
4. Make sure billing is set up: https://replicate.com/account/billing


## Cell 1: Installation & Imports


In [ ]:
# Install required packages (run once)
# !pip install replicate python-dotenv pillow ipython ipywidgets requests


In [ ]:
import os
import random
import json
from pathlib import Path
from io import BytesIO
from datetime import datetime

In [ ]:
import os
import random
import json
from pathlib import Path
from io import BytesIO
from datetime import datetime

from dotenv import load_dotenv
import replicate
from PIL import Image
from IPython.display import display, Image as IPImage, HTML
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed
import requests


## Cell 2: Configuration


In [ ]:
# Load environment variables from .env file
load_dotenv()

# Get API token
REPLICATE_API_TOKEN = os.getenv('REPLICATE_API_TOKEN')

if not REPLICATE_API_TOKEN:
    raise ValueError(
        "REPLICATE_API_TOKEN not found! "
        "Please create a .env file with your API token. "
        "Get your token from: https://replicate.com/account/api-tokens"
    )

# Initialize Replicate client
client = replicate.Client(api_token=REPLICATE_API_TOKEN)

# Model identifier
MODEL_ID = "recraft-ai/recraft-v3"

# Paths configuration
IMAGES_DIR = Path("Images")
OUTPUTS_DIR = Path("outputs")
OUTPUTS_DIR.mkdir(exist_ok=True)

print("✓ Configuration loaded successfully!")
print(f"✓ Images directory: {IMAGES_DIR.absolute()}")
print(f"✓ Outputs directory: {OUTPUTS_DIR.absolute()}")


## Cell 3: Image Loading Functions


In [ ]:
def load_all_images(images_dir=IMAGES_DIR):
    """Load all PNG files from the Images directory."""
    image_files = sorted(list(images_dir.glob("*.png")))
    if not image_files:
        print(f"No PNG files found in {images_dir}")
        return []
    
    images_data = []
    for img_path in image_files:
        try:
            img = Image.open(img_path)
            images_data.append({
                "path": img_path,
                "name": img_path.name,
                "image": img,
                "size": img.size
            })
        except Exception as e:
            print(f"Error loading {img_path.name}: {e}")
    
    return images_data

def display_images(images_data, max_cols=3):
    """Display all loaded images in a grid."""
    if not images_data:
        print("No images to display")
        return
    
    from IPython.display import display as ipydisplay
    
    print(f"Found {len(images_data)} image(s):\n")
    for img_data in images_data:
        print(f"• {img_data['name']} - {img_data['size']}")
        display(img_data['image'])

# Load and display all images
all_images = load_all_images()
if all_images:
    display_images(all_images)


In [ ]:
def get_image_by_name(name, images_data=None):
    """Get a specific image by filename."""
    if images_data is None:
        images_data = load_all_images()
    
    for img_data in images_data:
        if img_data['name'] == name or name in img_data['name']:
            return img_data
    
    return None

# Example: Get a specific image
# selected_image = get_image_by_name("Artboard 1.png")
# if selected_image:
#     display(selected_image['image'])


## Cell 4: Core Generation Functions


In [ ]:
def generate_image(prompt, size="1365x1024", seed=None, progress_callback=None):
    """
    Generate an image using Recraft V3 model via Replicate API.
    
    Parameters:
    -----------
    prompt : str
        Text prompt describing the image to generate
    size : str
        Image size in format "WIDTHxHEIGHT" (default: "1365x1024")
    seed : int, optional
        Random seed for reproducibility
    progress_callback : callable, optional
        Function to call with progress updates
    
    Returns:
    --------
    PIL.Image
        Generated image
    """
    if progress_callback:
        progress_callback("Starting generation...")
    
    # Prepare input parameters
    input_params = {
        "prompt": prompt,
        "size": size
    }
    
    if seed is not None:
        input_params["seed"] = seed
    
    try:
        if progress_callback:
            progress_callback("Calling Replicate API...")
        
        # Run the model
        output = client.run(MODEL_ID, input=input_params)
        
        if progress_callback:
            progress_callback("Downloading generated image...")
        
        # Get the image URL and download
        if hasattr(output, 'url'):
            image_url = output.url(0)
        elif isinstance(output, list) and len(output) > 0:
            image_url = output[0]
        elif isinstance(output, dict) and 'output' in output:
            image_url = output['output'][0] if isinstance(output['output'], list) else output['output']
        else:
            image_url = str(output)
        
        # Download the image
        response = requests.get(image_url)
        response.raise_for_status()
        
        # Convert to PIL Image
        image = Image.open(BytesIO(response.content))
        
        if progress_callback:
            progress_callback("✓ Generation complete!")
        
        return image
        
    except Exception as e:
        error_msg = f"Error generating image: {str(e)}"
        if progress_callback:
            progress_callback(error_msg)
        raise Exception(error_msg)

# Test the function (optional)
# test_image = generate_image("a modern minimalist logo design", progress_callback=print)
# display(test_image)


In [ ]:
def save_generated_image(image, prompt, seed=None, size="1365x1024", base_name="generated"):
    """Save a generated image with metadata."""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    seed_str = f"_seed{seed}" if seed is not None else ""
    filename = f"{base_name}_{timestamp}{seed_str}.webp"
    filepath = OUTPUTS_DIR / filename
    
    # Save image
    image.save(filepath, "WEBP")
    
    # Save metadata
    metadata = {
        "prompt": prompt,
        "size": size,
        "seed": seed,
        "timestamp": timestamp,
        "filename": filename
    }
    
    metadata_path = filepath.with_suffix(".json")
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"✓ Saved: {filepath}")
    print(f"✓ Metadata: {metadata_path}")
    
    return filepath, metadata


## Cell 5: Random Generation Interface


In [ ]:
# Predefined style prompts for branding/icon variations
STYLE_PROMPTS = [
    "a modern minimalist logo design with clean lines",
    "a colorful vibrant icon with bold shapes",
    "a sleek futuristic branding element with geometric patterns",
    "an elegant classic logo with refined details",
    "a playful creative icon design with organic shapes",
    "a professional corporate logo with structured elements",
    "a retro vintage branding style with nostalgic elements",
    "a minimalist abstract icon with geometric simplicity",
    "a dynamic energetic logo with flowing lines",
    "a sophisticated luxury branding with ornate details"
]

# Size options
SIZE_OPTIONS = [
    "1365x1024",
    "1024x1024",
    "1024x768",
    "768x1024"
]

def random_generate(num_images=1, base_prompt=None, save=True):
    """
    Generate random variations of branding/icon designs.
    
    Parameters:
    -----------
    num_images : int
        Number of images to generate
    base_prompt : str, optional
        Base prompt to combine with random styles. If None, uses random style only.
    save : bool
        Whether to save generated images
    
    Returns:
    --------
    list
        List of generated PIL Images
    """
    generated_images = []
    
    for i in range(num_images):
        print(f"\n[{i+1}/{num_images}] Generating random variation...")
        
        # Randomly select style and parameters
        style_prompt = random.choice(STYLE_PROMPTS)
        size = random.choice(SIZE_OPTIONS)
        seed = random.randint(1, 1000000)
        
        # Combine base prompt if provided
        if base_prompt:
            prompt = f"{base_prompt}, {style_prompt}"
        else:
            prompt = style_prompt
        
        print(f"  Prompt: {prompt}")
        print(f"  Size: {size}")
        print(f"  Seed: {seed}")
        
        try:
            image = generate_image(prompt, size=size, seed=seed, progress_callback=lambda x: print(f"    {x}"))
            
            # Display image
            display(image)
            
            # Save if requested
            if save:
                save_generated_image(image, prompt, seed=seed, size=size, base_name="random")
            
            generated_images.append({
                "image": image,
                "prompt": prompt,
                "size": size,
                "seed": seed
            })
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    return generated_images

# Example: Generate 3 random variations
# random_images = random_generate(num_images=3)


## Cell 6: Manual Control Interface


In [ ]:
def manual_generate(prompt, size="1365x1024", seed=None, save=True, display_img=True):
    """
    Generate an image with manual control over all parameters.
    
    Parameters:
    -----------
    prompt : str
        Text prompt describing the desired image
    size : str
        Image size in format "WIDTHxHEIGHT"
    seed : int, optional
        Random seed for reproducibility. If None, uses random seed.
    save : bool
        Whether to save the generated image
    display_img : bool
        Whether to display the generated image
    
    Returns:
    --------
    dict
        Dictionary containing image, prompt, size, and seed
    """
    if seed is None:
        seed = random.randint(1, 1000000)
    
    print(f"Generating with manual parameters...")
    print(f"  Prompt: {prompt}")
    print(f"  Size: {size}")
    print(f"  Seed: {seed}")
    
    try:
        image = generate_image(prompt, size=size, seed=seed, progress_callback=lambda x: print(f"  {x}"))
        
        if display_img:
            display(image)
        
        if save:
            filepath, metadata = save_generated_image(image, prompt, seed=seed, size=size, base_name="manual")
        
        result = {
            "image": image,
            "prompt": prompt,
            "size": size,
            "seed": seed
        }
        
        if save:
            result["filepath"] = filepath
            result["metadata"] = metadata
        
        return result
        
    except Exception as e:
        print(f"✗ Error: {e}")
        raise

# Example: Manual generation
# result = manual_generate(
#     prompt="a modern minimalist icon design with blue and white colors",
#     size="1024x1024",
#     seed=42,
#     save=True
# )


In [ ]:
# Interactive widget for manual generation (optional)
def create_generation_widget():
    """Create an interactive widget for manual image generation."""
    prompt_widget = widgets.Textarea(
        value="a modern minimalist logo design",
        placeholder="Enter your prompt here...",
        description="Prompt:",
        layout=widgets.Layout(width='100%', height='80px')
    )
    
    size_widget = widgets.Dropdown(
        options=SIZE_OPTIONS,
        value="1365x1024",
        description="Size:"
    )
    
    seed_widget = widgets.IntText(
        value=None,
        description="Seed (optional):",
        placeholder="Leave blank for random"
    )
    
    save_widget = widgets.Checkbox(
        value=True,
        description="Save image"
    )
    
    generate_button = widgets.Button(
        description="Generate Image",
        button_style="success",
        icon="image"
    )
    
    output_widget = widgets.Output()
    
    def on_generate_click(b):
        with output_widget:
            output_widget.clear_output()
            seed = seed_widget.value if seed_widget.value else None
            try:
                manual_generate(
                    prompt=prompt_widget.value,
                    size=size_widget.value,
                    seed=seed,
                    save=save_widget.value
                )
            except Exception as e:
                print(f"Error: {e}")
    
    generate_button.on_click(on_generate_click)
    
    return widgets.VBox([
        widgets.HTML("<h3>Manual Image Generation</h3>"),
        prompt_widget,
        widgets.HBox([size_widget, seed_widget]),
        save_widget,
        generate_button,
        output_widget
    ])

# Uncomment to use interactive widget
# display(create_generation_widget())


## Cell 7: Batch Processing


In [ ]:
def batch_generate(prompts, size="1365x1024", seeds=None, base_name="batch"):
    """
    Generate multiple images from a list of prompts.
    
    Parameters:
    -----------
    prompts : list of str
        List of prompts for generation
    size : str
        Image size (same for all)
    seeds : list of int, optional
        Seeds for each prompt. If None, uses random seeds.
    base_name : str
        Base name for saved files
    
    Returns:
    --------
    list
        List of result dictionaries
    """
    if seeds is None:
        seeds = [random.randint(1, 1000000) for _ in prompts]
    elif len(seeds) != len(prompts):
        raise ValueError("Number of seeds must match number of prompts")
    
    results = []
    
    for i, prompt in enumerate(prompts):
        print(f"\n[{i+1}/{len(prompts)}] Processing: {prompt[:50]}...")
        
        try:
            result = manual_generate(
                prompt=prompt,
                size=size,
                seed=seeds[i],
                save=True,
                display_img=False
            )
            results.append(result)
            
        except Exception as e:
            print(f"  ✗ Failed: {e}")
            results.append({"error": str(e), "prompt": prompt})
    
    print(f"\n✓ Batch processing complete! {len([r for r in results if 'error' not in r])}/{len(prompts)} successful")
    
    return results

# Example: Batch generation
# prompts = [
#     "a modern minimalist logo",
#     "a colorful vibrant icon",
#     "a sleek futuristic branding element"
# ]
# results = batch_generate(prompts)


## Cell 8: Utilities


In [ ]:
def compare_images(original_path, generated_image, original_label="Original", generated_label="Generated"):
    """Display original and generated images side by side."""
    from IPython.display import display as ipydisplay
    
    original = Image.open(original_path)
    
    # Resize images to same height for comparison
    target_height = max(original.height, generated_image.height)
    
    original_resized = original.resize(
        (int(original.width * target_height / original.height), target_height),
        Image.Resampling.LANCZOS
    )
    
    generated_resized = generated_image.resize(
        (int(generated_image.width * target_height / generated_image.height), target_height),
        Image.Resampling.LANCZOS
    )
    
    # Create comparison image
    comparison = Image.new('RGB', (original_resized.width + generated_resized.width, target_height))
    comparison.paste(original_resized, (0, 0))
    comparison.paste(generated_resized, (original_resized.width, 0))
    
    display(comparison)
    print(f"{original_label} | {generated_label}")

def list_generated_images(outputs_dir=OUTPUTS_DIR):
    """List all generated images in the outputs directory."""
    image_files = sorted(list(outputs_dir.glob("*.webp")))
    
    if not image_files:
        print("No generated images found.")
        return []
    
    print(f"Found {len(image_files)} generated image(s):\n")
    
    for img_path in image_files:
        metadata_path = img_path.with_suffix(".json")
        
        print(f"• {img_path.name}")
        
        if metadata_path.exists():
            with open(metadata_path, 'r') as f:
                metadata = json.load(f)
            print(f"  Prompt: {metadata.get('prompt', 'N/A')}")
            print(f"  Seed: {metadata.get('seed', 'N/A')}")
            print(f"  Size: {metadata.get('size', 'N/A')}")
            print(f"  Timestamp: {metadata.get('timestamp', 'N/A')}")
        print()
    
    return image_files

# List all generated images
# list_generated_images()


## Quick Start Examples

### Random Generation
```python
# Generate 3 random variations
random_images = random_generate(num_images=3, base_prompt="logo design", save=True)
```

### Manual Generation
```python
# Generate with specific parameters
result = manual_generate(
    prompt="a modern minimalist icon with blue and white colors",
    size="1024x1024",
    seed=42
)
```

### Batch Generation
```python
# Generate multiple variations
prompts = [
    "a modern minimalist logo",
    "a colorful vibrant icon",
    "a sleek futuristic branding"
]
results = batch_generate(prompts, size="1024x1024")
```

### Interactive Widget
```python
# Use the interactive widget for easy generation
display(create_generation_widget())
```
